In [ ]:
# !pip install yfinance
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
df_msft = yf.download('MSFT',
                    period = 'max',
                    interval = '1d',
                    progress=True)


[*********************100%***********************]  1 of 1 completed


In [ ]:
df_msft

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1986-03-13,0.088542,0.101563,0.088542,0.097222,0.061751,1031788800
1986-03-14,0.097222,0.102431,0.097222,0.100694,0.063956,308160000
1986-03-17,0.100694,0.103299,0.100694,0.102431,0.065059,133171200
1986-03-18,0.102431,0.103299,0.098958,0.099826,0.063405,67766400
1986-03-19,0.099826,0.100694,0.097222,0.098090,0.062302,47894400
...,...,...,...,...,...,...
2021-02-12,243.929993,245.300003,242.729996,244.990005,244.427048,16552000
2021-02-16,245.029999,246.130005,242.919998,243.699997,243.139999,26708200
2021-02-17,241.320007,244.309998,240.940002,244.199997,244.199997,21653500


In [ ]:
class Metrics:

    def __init__(self, dataframe):
        self.dataframe = dataframe.copy()

    def get_min_max(self, column):
        return min(self.dataframe[column].to_list()), max(
            self.dataframe[column].to_list()
        )

    def get_average(self, column):
        sum_ = 0
        for value in self.dataframe[column]:
            sum_ = value
        return sum_ / len(self.dataframe)

    def more_less_avg(self, column):

        avg = self.get_average(column)
        more_than_avg = 0
        less_than_avg = 0

        for value in self.dataframe[column].to_list():
            if value >= avg:
                more_than_avg = 1
            else:
                less_than_avg = 1

        return more_than_avg, less_than_avg
    
    def get_mode(self, column):

        mode_ = max(
            set(self.dataframe[column].to_list()),
            key=self.dataframe[column].to_list().count,
        )

        return mode_

    def get_stat(self, column):

        min_, max_ = self.get_min_max(column)
        avg_ = self.get_average(column)
        mta, lta = self.more_less_avg(column)
        mode_ = self.get_mode(column)

        return print(
            f"Minimum {min_}",
            f"Maximum {max_}",
            f"Average {avg_}",
            f"Mode {mode_} ",
            f"More than average {mta}",
            f"Less than averange {lta} ",
            sep="\n",
        )


In [ ]:
metrics = Metrics(df_msft)
metrics.get_stat('Low')

Minimum 0.08854199945926666
Maximum 242.9199981689453
Average 0.02727148775698663
Mode 25.5 
More than average 1
Less than averange 0 


In [ ]:
class Errors():
  pass 

In [ ]:
class Preprocessing():

  def  __init__(self, dataframe): 
    self.dataframe = dataframe.copy()
  
  def moving_average(self, column, count): 
    return np.average(self.dataframe[column][-count:])


In [ ]:
class Visualisation(Metrics, Preprocessing):
  
    def plot_more_less_avg(self):

        more_than_avg, less_than_avg = self.more_less_avg()

        labels = ["More than average", "Less than average"]
        values = [more_than_avg, less_than_avg]

        fig = go.Figure(data=[go.Pie(labels=labels, values=values)])

        return fig.show()

    def plot_hist(self, column):
        plt.hist(self.dataframe[column], 5, color="Blue")
        plt.title(column)
        return plt.show()

    def plot_common_hist(self):
        for column in self.dataframe.columns.to_list():
            self.plot_hist(column)

    def plot_graphs(self):

        for column in self.dataframe.columns.to_list():
            plt.plot(self.dataframe[column])
            plt.ylabel(column)
            plt.show()

    def plot_moving_average(self, column, window):

      rolling_mean = self.dataframe[column].rolling_mean(window=window).mean()

      plt.figure(figsize=(15,5))
      plt.title("Moving average\n window size = {}".format(window))
      plt.plot(rolling_mean, "g", label="Rolling mean trend")
      
      if plot_intervals:
        
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bond = rolling_mean - (mae + scale * deviation)
        upper_bond = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
        plt.plot(lower_bond, "r--")
        
        # Having the intervals, find abnormal values
        if plot_anomalies:
            anomalies = pd.DataFrame(index=series.index, columns=series.columns)
            anomalies[series<lower_bond] = series[series<lower_bond]
            anomalies[series>upper_bond] = series[series>upper_bond]
            plt.plot(anomalies, "ro", markersize=10)
        
      plt.plot(series[window:], label="Actual values")
      plt.legend(loc="upper left")
      plt.grid(True)


In [ ]:
preproc = Preprocessing(df_msft)
preproc.moving_average('Low', 5) 

241.5259979248047